<a href="https://colab.research.google.com/github/IgorVaryvoda/Sirv-auto-tag-images/blob/main/Auto_alt_tags_Sirv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Image alt tags are important for both accessibility and SEO. But writing alt tags and adding them to images can be quite a chore. What if there was a way to automate this process?

Luckily, image recognition services like Azure's Computer Vision have come a long way. Let's leverage the power of AI and Sirv to automatically add alt tags to your images.

[Read the full article here ](https://sirv.com/help/articles/auto-alt-tags-ai)

Follow these steps:

# 1. Install Azure SDK



In [1]:
!pip install azure-cognitiveservices-vision-computervision
# Download the demo images file (VM filesystem isn't persistent)
!wget -q https://sirv.sirv.com/website/screenshots/auto-alt-tags/images.txt

     |████████████████████████████████| 85 kB 3.7 MB/s 
     |████████████████████████████████| 41 kB 630 kB/s 


#2. Add API keys

## 2.1 Azure key
Follow the instructions <a href="https://sirv.com/help/articles/auto-alt-tags-ai#cv">here</a>
##2.2 Sirv keys
[Follow these instructions](https://sirv.com/help/articles/auto-alt-tags-ai#get-sirv-rest-api-keys)

In [ ]:
#@title 2.3 Add your API keys here
#@markdown Azure key

subscription_key = ''  #@param {type: "string"}
#@markdown Sirv keys
sirv_id = '' #@param {type: "string"}
sirv_secret = '' #@param {type: "string"}


# Add your images 
Add images (one image per line) to /content/images.txt

Learn how to get them here - https://sirv.com/help/articles/auto-alt-tags-ai#adding-image-alt-tags-in-bulk

# Run the code

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import json
import requests
import time
from urllib.parse import urlparse

#Create Azure client
endpoint = "https://we.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

# Get Sirv auth token. API reference - https://apidocs.sirv.com/#connect-to-sirv-api
payload = {
  'clientId': sirv_id,
  'clientSecret': sirv_secret
}
#We're opening the csv file and looping through each image in the file
with open('images.txt') as f:
  for image in f:
    headers = {'content-type': 'application/json', "Origin": "*"}
    response = requests.request('POST', 'https://api.sirv.com/v2/token', data=json.dumps(payload), headers=headers)
    token = response.json()['token']

    # Create a Computer Vision client
    description_results = computervision_client.describe_image(image)

    # Get the captions (descriptions) from the response, with confidence level
    print("Description of remote image: ")
    if (len(description_results.captions) == 0):
      print("No description detected.")
    else:
      for caption in description_results.captions:
        print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))
        description = '{"description": "'+ caption.text + '"}'
        '''
        Update the image description in Sirv.
        API reference - https://apidocs.sirv.com/#set-meta-description
        '''
        params = {"filename": urlparse(image).path.replace('\n', '')}
        headers = {
          'content-type': "application/json",
          "Origin": "*",
          'authorization': 'Bearer %s' % token
        }
        print('Updating the image description in Sirv')
        response = requests.request('POST', 'https://api.sirv.com/v2/files/meta/description', data=description.encode('utf-8'), headers=headers, params=params)
        print(response)
        print('4 second pause')
        time.sleep(4)

Description of remote image: 
'a close up of a leopard' with confidence 52.22%
Updating the image description in Sirv
<Response [200]>
4 second pause
Description of remote image: 
'a pair of red headphones' with confidence 51.82%
Updating the image description in Sirv
<Response [200]>
4 second pause
